In [1]:
%env MODEL_VERSION=1

env: MODEL_VERSION=1


In [2]:
# cmd to check gpu process
!nvidia-smi

Sun Sep 11 10:40:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"

In [4]:
!pip install -q tf-models-official==2.7.0

In [5]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [6]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [7]:
import pandas as pd

data = pd.read_csv('../gdrive/MyDrive/bert-chatbot/sample.csv')

data = data[data['intent']!='FALLBACK']
data = data[data['intent']!='FORBIDDEN']
data = data[~data['intent'].str.startswith("OTHERS_")]

df = pd.DataFrame(data)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['intent'])

num_of_classes = len(le.classes_)

In [8]:
import json

f = open('../gdrive/MyDrive/bert-chatbot/reply.json')
response = json.load(f)
print(response['GENDER'])

[{'en': "I don't know my gender yet, but I was created by a male developer.", 'zh': '我仲未知我係男定女，但我嘅開發者係男人嚟。', 'fr': 'Je ne sais pas encore, mais mon développeur est un homme.'}]


In [9]:
model_dir='/gdrive/MyDrive/bert-chatbot/model/' + os.getenv("MODEL_VERSION") + '/saved_model'

reloaded = tf.saved_model.load(model_dir)

In [12]:
print("response language: en\n")
lang = 'en'
query = ["你好!", "hello", "琴日入完tqqq 今日即跌", "你辦公時間是甚麼?", "you perform bad"]

predictions = reloaded(query)

for i, pred in enumerate(predictions):
  predictedIntent = le.classes_[tf.argmax(pred).numpy()]
  print("Query: {}".format(query[i]))
  print("Response: {} [Intention predicted: {}]".format(response.get(predictedIntent)[0][lang], predictedIntent))

response language: en

Query: 你好!
Response: Hi, how is your day? [Intention predicted: GREETINGS]
Query: hello
Response: Hi, how is your day? [Intention predicted: GREETINGS]
Query: 琴日入完tqqq 今日即跌
Response: I lost a lot on stock, sooner I won't even have money to pay the rent cost on AWS EC2... which means you may not reach me later. [Intention predicted: INVESTMENT]
Query: 你辦公時間是甚麼?
Response: As long as the server keeps running, I work 7/24. So just reach me anytime you are free. [Intention predicted: OFFICE_HOUR]
Query: you perform bad
Response: Sorry sorry, I know I am not smart enough. Maybe I will improve in the future with more training data. [Intention predicted: APPRAISAL_BAD]
